In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Answer generation with LSTM from context and prompt

In [ ]:
data1 = "/kaggle/input/sci-or-not-sci-hypthesis-testing-pack/6000_all_categories_questions_with_excerpts.csv"
data2="/kaggle/input/sci-or-not-sci-hypthesis-testing-pack/6000_wiki_en_sci_questions_with_excerpts.csv"
data = "/kaggle/input/kaggle-llm-science-exam/train.csv"

In [ ]:
# df = pd.concat([pd.read_csv(data1),pd.read_csv(data2)])
# df.reset_index(drop=True, inplace=True)
df = pd.read_csv(data1)[:2000]

In [ ]:
def concat_all(x):
    ans = x[x['answer']]
    concat_str = x['wikipedia_excerpt'].split(':')[1].strip() + '. '+ x['prompt'] + ' <ANS>' + str(ans)
    return concat_str

In [ ]:
df['concat_prompt_ans_exc'] = df.apply(lambda x: concat_all(x), axis = 1)

In [ ]:
import random

def generate_random_numbers(count,lgth):
    random_numbers = [random.randint(0, lgth) for _ in range(count)]
    return random_numbers

# Example: Generate a list of 10 random numbers
lgth = len(df)-1
random_list = generate_random_numbers(int(0.2*len(df)),lgth)
df['test']=False
df['test'].loc[random_list]=True

## Tmp

In [ ]:
df.shape

In [ ]:
import string
import regex as re
def clean_txt(article):
  article = re.sub(r'[^\w\s]', '', article)
  article = re.sub(r'[_]', '', article)
  tokens = article.split()
  tokens = [t for t in tokens if t not in string.punctuation]
#   tokens = [t for t in tokens if t.isalpha()]
  tokens = [t.lower() for t in tokens]
  return tokens
articles = []
description = df['concat_prompt_ans_exc']

for i in range(len(description)):
  art = clean_txt(description[i])
  article = []
  for word in art:
    article.append(word)
  articles.append(article)

In [ ]:
articles_flatten = [k for a in articles for k in a ]

In [ ]:
print('total Tokens',len(articles_flatten)) #total Tokens 11503
print('total Tokens',len(set(articles_flatten))) #total Tokens 1918

In [ ]:
maxlen = 0
for i in articles:
    if maxlen<len(i):
        maxlen = len(i)

In [ ]:
length = 50
seq = []
for i in articles:
    if length >= len(i):
        lines = ' '.join(i)
        seq.append(lines)
    else:
        for j in range(length, len(i)+1):
            sequence = i[j-length:j]
            lines = ' '.join(sequence)
            seq.append(lines)
print('Total Sequence',len(seq)) #Total Sequence 11497

## Back

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(seq)
tokens = tokenizer.texts_to_sequences(seq)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size) # 1919

In [ ]:
maxlen = 0
for i in tokens:
    if maxlen<len(i):
        maxlen = len(i)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded = pad_sequences(tokens,maxlen=maxlen,truncating='pre')

In [ ]:
X,y = encoded[:,:-1],encoded[:,-1]

In [ ]:
from keras.utils import to_categorical

In [ ]:
y = to_categorical(y,num_classes=vocab_size)

In [ ]:
y.shape

In [ ]:
embeddings_index = dict()
f = open('/kaggle/input/glove6b/glove.6B.200d.txt',mode='rt',encoding='utf-8')
import numpy as np
for line in f:
    values = line.split()
    words = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[words] = coefs
f.close()
print('Loaded word vectors',len(embeddings_index)) 
#Loaded word vectors 400000

In [ ]:
embedding_matrix = np.zeros((vocab_size,200))
for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if(embedding_vector is not None):
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
def build_model(vocab_size,seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size,200,weights=[embedding_matrix],trainable=False,input_length=seq_length))
  
  model.add(LSTM(100,return_sequences=True))
  model.add(LSTM(100))
  
  model.add(Dense(100, activation='relu'))
  model.add(Dense(vocab_size, activation='softmax'))
  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize defined model
  model.summary()
  return model

In [ ]:
# model = tf.keras.Sequential([
# tf.keras.layers.Embedding(input_dim = total_unique_words, output_dim=100, weights=[embeddings_matrix], input_length=max_seq_length-1, trainable=False),
# tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
# tf.keras.layers.Dropout(0.2), 
# tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
# tf.keras.layers.Dropout(0.2),
# tf.keras.layers.Dense(128, activation='relu'),
# tf.keras.layers.Dense(total_unique_words , activation='softmax')])
# model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(x_values, y_values, epochs=120, validation_split=0.2, verbose=1, batch_size=256)


In [ ]:
model = build_model(vocab_size,maxlen-1)

In [ ]:
model.fit(X,y,batch_size=8,epochs=25)

In [ ]:
model

In [ ]:
# Installing offline dependencies
!pip install -U --no-deps /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install -U --no-deps /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl
# !pip install -U --no-deps /kaggle/input/optimum-113/optimum-1.13.2-py3-none-any.whl

In [ ]:
class SentenceTransformer:
    def __init__(self, checkpoint, device="cuda:1"):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def transform(self, batch):
        tokens = self.tokenizer(batch["text"], truncation=True, padding=True, return_tensors="pt", max_length=MAX_SEQ_LEN)
        return tokens.to(self.device)  

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=1):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader

        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [ ]:
from time import time
MODEL_PATH = "/kaggle/input/bge-small-faiss/"
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModel
import gc

In [ ]:
# For RAG
import faiss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset
MAX_SEQ_LEN = 512
import torch
import ctypes
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

In [ ]:
df_test = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')

In [ ]:
NUM_TITLES = 5


In [ ]:
# Load embedding model
start = time()
print(f"Starting prompt embedding, t={time() - start :.1f}s")
model_faiss = SentenceTransformer(MODEL_PATH, device="cuda:1")

# Get embeddings of prompts
f = lambda row : " ".join([row["prompt"], str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"]), str(row["E"]))
inputs = df_test.apply(f, axis=1).values # better results than prompt only
prompt_embeddings = model_faiss.encode(inputs, show_progress_bar=False)

# Search closest sentences in the wikipedia index 
print(f"Loading faiss index, t={time() - start :.1f}s")
faiss_index = faiss.read_index(MODEL_PATH + '/faiss.index')
# faiss_index = faiss.index_cpu_to_all_gpus(faiss_index) # causes OOM, and not that long on CPU

print(f"Starting text search, t={time() - start :.1f}s")
search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

print(f"Starting context extraction, t={time() - start :.1f}s")
dataset = load_from_disk("/kaggle/input/all-paraphs-parsed-expanded")
for i in range(len(df_test)):
    df_test.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

# Free memory
faiss_index.reset()
del faiss_index, prompt_embeddings, model_faiss, dataset
clean_memory()
print(f"Context added, t={time() - start :.1f}s")

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def generate_seq(model,tokenizer,seq_length,seed_txt,n_words):
  in_txt = seed_txt
  result = list()
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_txt])[0]
    encoded = pad_sequences([encoded],maxlen=seq_length,truncating='pre')
    yhat = model.predict(encoded, verbose = None)
    yhat=np.argmax(yhat,axis=1)
    out_word = ''
    for word,index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    in_txt += ' '+out_word
    result.append(out_word)
  return ' '.join(result),in_txt

In [ ]:
def concat_prompt_cont(x):
    concat_str =  x['context'] + x['prompt'] 
    return concat_str

In [ ]:
df_test['concat_prompt_exc'] = df_test.apply(lambda x: concat_prompt_cont(x), axis = 1)

In [ ]:
df_train = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')

In [ ]:
df_test['answer']= df_train['answer']

In [ ]:
for text in df_test[:10].iterrows():
    text = text[1]
    input_pr = text['concat_prompt_exc']
    print(text['prompt'])
    ans_len = min([len(text['A'].split()),  len(text['B'].split()),len(text['C'].split()),len(text['D'].split()),len(text['E'].split())]) 
    generated,in_txt = generate_seq(model, tokenizer, 50, input_pr, ans_len)
    print('Predicted Answer:')
    print(generated)
    print('Correct Answer;')
    print(text[text['answer']])
    print('================')

In [ ]:
for text in df_test['concat_prompt_exc'][:10]:
    text = text[-195:]
    
    generated,in_txt = generate_seq(model, tokenizer, 196, text, 10)
    print('~After~')
    print(in_txt)
    print('================')